# Hyperparameter optimization

In this experiment the hyperparameters of the model will be optimized on cryptocurrency data. The original paper was released with optimized hyperparameters. But because they did not use cryptocurrency data, new experiments will be done for optimizing the model's hyperparameters on cryptocurrency data. This notebook is made to explain the parts of the code of the scripts. For this notebook the experiment on the learning rate is taken as an example.

## Imports

First the necessary modules en scripts are imported.

In [1]:
from matplotlib.font_manager import json_dump
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import sys
import math
from preprocess_data import preprocess

WORKDIR_PATH = os.getcwd() + "/../../"
sys.path.insert(1, WORKDIR_PATH)

from base.train_scinet import train_scinet

## Preprocessing

In this part the data is being loaded and preprocessed. At first the settings of the preprocessing are defined.

In [2]:
data_format=["open","high","low","close","Volume BTC","Volume USDT","tradecount"]
                    
fraction_used = 1
train_frac = 0.6
val_frac = 0.2
test_frac = 0.2

X_LEN = 48
Y_LEN = 24
RANDOM_SEED = 4321#None
OVERLAPPING = True
STANDARDIZE = True

standardization_settings = {'per_sample': True,
                            'leaky': False,
                            'mode': 'log', #only if per sample is false, choose from log, sqrt or lin
                            'sqrt_val': 2, #of course only if mode is sqrt
                            'total mean': [],
                            'total std': []}

Then the dataset is loaded, splitted in samples and train/test sets, and the samples are being normalized.

In [3]:
pairs = ["HUFL", "HULL", "MUFL", "MULL", "LUFL", "LULL", "OT"]

#df = pd.read_csv(os.path.realpath(__file__) + f"/../data/Data_preprocessed/ETTh1.csv").dropna()
df = pd.read_csv(os.getcwd() + "/data/Binance_BTCUSDT_minute.csv").dropna()
df = df.swapaxes("index", "columns")

data = {} 
for idx, pair in enumerate(pairs):
    data[pair] = df.iloc[idx]
 
results = preprocess(   data = data, 
                        symbols = pairs,
                        data_format = data_format,
                        fraction = fraction_used,
                        train_frac = train_frac,
                        val_frac = val_frac,
                        test_frac = test_frac,
                        X_LEN = X_LEN,
                        Y_LEN = Y_LEN,
                        OVERLAPPING = OVERLAPPING,
                        STANDARDIZE = STANDARDIZE,
                        standardization_settings = standardization_settings
                        )

Starting data preprocessing...
   48740.22  48745.96  48727.47  48727.47.1   2.27206  110730.9135  136.0
0  48763.11  48763.12  48736.70    48736.73   5.33108  259880.1205  427.0
1  48778.58  48778.58  48750.37    48763.12   6.87389  335219.0368  389.0
2  48760.37  48778.58  48746.39    48778.58  10.58951  516291.2896  425.0
3  48799.99  48800.00  48756.93    48760.37  12.24525  597357.8390  535.0
4  48795.99  48800.00  48795.99    48800.00   7.55759  368810.1891  423.0 (49997, 7)
Making train/validation/test splits...
Making samples...


100%|███████████████████████████████████| 29926/29926 [00:23<00:00, 1251.49it/s]
/Users/lindsayspoor/Library/Mobile Documents/com~apple~CloudDocs/Documents/Studiedocumenten/2021-2022/ADL/SCINet_repo/exp/hyperparams/preprocess_data.py:138: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  samples = np.array(samples)


Making samples...


100%|█████████████████████████████████████| 9927/9927 [00:07<00:00, 1325.98it/s]


Making samples...


100%|█████████████████████████████████████| 9928/9928 [00:08<00:00, 1206.75it/s]


Making X-y splits...


## Training

Then the model can be trained. At first the hyperparameters which are not being optimized are defined.

In [4]:
EPOCHS = 10
BATCH_SIZE = 8
HID_SIZE = 4
NUM_LEVELS = 3
KERNEL_SIZE = 5
DROPOUT = 0.5
PROBABILISTIC = False

Then some values of the parameter to be tuned (in this case the learning rate) are defined. For each value of this parameter a model is trained and it's performance on the validation set is saved for plotting later on.

In [5]:
LEARNING_RATES = [0.1, 0.01, 0.001, 0.0001]

train_losses = np.zeros((len(LEARNING_RATES), EPOCHS))
val_losses = np.zeros((len(LEARNING_RATES), EPOCHS))
for idx, LEARNING_RATE in enumerate(LEARNING_RATES):

    model, history, X_train , y_train, X_val, y_val, X_test, y_test = train_scinet( X_train = results["X_train"].astype('float32'),
                                                                                    y_train = results["y_train"].astype('float32'),
                                                                                    X_val = results["X_val"].astype('float32'),
                                                                                    y_val = results["y_val"].astype('float32'),
                                                                                    X_test = results["X_test"].astype('float32'),
                                                                                    y_test = results["y_test"].astype('float32'),
                                                                                    epochs = EPOCHS,
                                                                                    batch_size = BATCH_SIZE,
                                                                                    X_LEN = X_LEN,
                                                                                    Y_LEN = [Y_LEN],
                                                                                    output_dim = [results["X_train"].shape[2]],
                                                                                    selected_columns = None,
                                                                                    hid_size= HID_SIZE,
                                                                                    num_levels= NUM_LEVELS,
                                                                                    kernel = KERNEL_SIZE,
                                                                                    dropout = DROPOUT,
                                                                                    loss_weights= [1],
                                                                                    learning_rate = LEARNING_RATE,
                                                                                    probabilistic = PROBABILISTIC)

    train_loss = history.history['loss']
    train_losses[idx] = train_loss

    val_loss = history.history['val_loss']
    val_losses[idx] = val_loss
    
    model.save(f'saved_models/model_learning_rate_{LEARNING_RATE}')

===========================[SCINET]=====================================
Initializing training with data:
X_train: (29926, 48, 7), y_train: (29926, 24, 7)
X_val: (9927, 48, 7), y_val: (9927, 24, 7)
X_test: (9928, 48, 7), y_test: (9928, 24, 7)
Building model...


2022-06-23 11:41:04.781569: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 7)]           0         
_________________________________________________________________
Block_0 (SCINet)             (None, 24, 7)             97332     
Total params: 97,332
Trainable params: 97,332
Non-trainable params: 0
_________________________________________________________________
None
Is null X: 0
Is null y: 0
Epoch 1/10


2022-06-23 11:41:07.055162: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


3741/3741 [==============================] - 126s 29ms/step - loss: 1.5101 - val_loss: 1.5656
Epoch 2/10
3741/3741 [==============================] - 97s 26ms/step - loss: 1.5497 - val_loss: 1.5485
Epoch 3/10
3741/3741 [==============================] - 99s 27ms/step - loss: 1.5736 - val_loss: 1.5277
Epoch 4/10
3741/3741 [==============================] - 98s 26ms/step - loss: 1.5584 - val_loss: 1.4476
Epoch 5/10
3741/3741 [==============================] - 103s 28ms/step - loss: 1.5129 - val_loss: 1.5305
Epoch 6/10
3741/3741 [==============================] - 103s 27ms/step - loss: 1.5693 - val_loss: 1.6152
Epoch 7/10
3741/3741 [==============================] - 95s 25ms/step - loss: 1.6144 - val_loss: 1.6312
Epoch 8/10
3741/3741 [==============================] - 85s 23ms/step - loss: 1.7115 - val_loss: 1.7257
Epoch 9/10
3741/3741 [==============================] - 102s 27ms/step - loss: 1.6596 - val_loss: 1.5676
Epoch 10/10
3741/3741 [==============================] - 104s 28ms/step

...

INFO:tensorflow:Assets written to: saved_models/model_learning_rate_0.001/assets


INFO:tensorflow:Assets written to: saved_models/model_learning_rate_0.001/assets


===========================[SCINET]=====================================
Initializing training with data:
X_train: (29926, 48, 7), y_train: (29926, 24, 7)
X_val: (9927, 48, 7), y_val: (9927, 24, 7)
X_test: (9928, 48, 7), y_test: (9928, 24, 7)
Building model...
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 48, 7)]           0         
_________________________________________________________________
Block_0 (SCINet)             (None, 24, 7)             97332     
Total params: 97,332
Trainable params: 97,332
Non-trainable params: 0
_________________________________________________________________
None
Is null X: 0
Is null y: 0
Epoch 1/10
3741/3741 [==============================] - 140s 31ms/step - loss: 1.7047 - val_loss: 1.2505
Epoch 2/10
3741/3741 [==============================] - 113s 30ms/step - loss: 1.1882 - val_loss: 1.1381
Epoch 3/10
3741

INFO:tensorflow:Assets written to: saved_models/model_learning_rate_0.0001/assets


INFO:tensorflow:Assets written to: saved_models/model_learning_rate_0.0001/assets


## Plotting

Here the performance of each model on the validation set is compared using a plot. The hyperparamer of the model with the lowest loss in the validation set can be selected as the optimal value.

In [7]:

from utils.plotting import plot_barplot


hyperparameter_type='LearningRate'



plot_barplot(LEARNING_RATES, val_losses, hyperparameter_type)

ModuleNotFoundError: No module named 'utils.plotting'